In [1]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import cv2
import threading
import time
from playsound import playsound
import numpy as np
from keras.utils import img_to_array
from collections import deque
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import time
import csv
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

C:\Users\2002n\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load YOLOv8 model
yolo_model = YOLO('E:/Download Till 27-07-2024/Downloads/yolov8_model.pt')

# Load ConvLSTM model
convlstm_model = load_model('D:/my_model.keras')

C:\Users\2002n\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
#better bounding box with good gui

# Parameters
sequence_len = 10
min_analysis_frames = 3
frame_size = (64, 64)
class_map = {0: 'opened_panel', 1: 'unauthorized_intervention', 
             2: 'overload_forklift', 3: 'unsafe_walkway'}
unsafe_labels_yolo = ['opened_panel', 'unauthorized_intervention', 
                     'overload_forklift', 'unsafe_walkway']

# Safety statistics
safety_stats = {
    'total_frames': 0,
    'safe_frames': 0,
    'unsafe_events': 0,
    'last_alert': None,
    'alert_count': 0
}

def is_unsafe_yolo(frame):
    results = yolo_model(frame)[0]
    class_ids = results.boxes.cls.int().tolist()
    confidences = results.boxes.conf.tolist()
    detected_classes = [results.names[i] for i in class_ids]
    return results, any(cls in unsafe_labels_yolo for cls in detected_classes)

def predict_unsafe_behavior(buffer):
    sequence = [cv2.resize(f, frame_size) for f in buffer]
    sequence = [img_to_array(f) / 255.0 for f in sequence]
    sequence = np.expand_dims(np.array(sequence), axis=0)
    preds = convlstm_model.predict(sequence, verbose=0)
    return np.argmax(preds)

class SafetyMonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Workplace Safety Monitoring")
        self.root.geometry("1280x860")
        self.root.configure(bg="#121420")

        # Header
        tk.Label(root, text="AI-Powered Workplace Monitoring", 
                font=("Arial", 20, "bold"), bg="#121420", fg="cyan").place(x=400, y=10)

        # Video display
        self.video_label = tk.Label(root, bg="#121420")
        self.video_label.place(x=40, y=60, width=720, height=480)

        # Status panel
        self.status_label = tk.Label(root, text="Status: Waiting for video...", 
                                   font=("Arial", 14), bg="#121420", fg="white")
        self.status_label.place(x=800, y=60)

        # Event log
        self.log_box = tk.Text(root, height=12, width=50, state='disabled', 
                             bg="#1e1e2f", fg="white", font=("Consolas", 10))
        self.log_box.place(x=800, y=100)

        # Buttons frame
        btns = tk.Frame(root, bg="#121420")
        btns.place(x=800, y=340)
        self.load_btn = tk.Button(btns, text="Load Video", command=self.load_video, 
                                bg="#2e2e4f", fg="white", width=25)
        self.load_btn.grid(row=0, column=0, pady=5)

        # Safety statistics dashboard
        stats_frame = tk.Frame(root, bg="#1e1e2f", bd=2, relief=tk.RIDGE)
        stats_frame.place(x=40, y=550, width=720, height=280)
        
        tk.Label(stats_frame, text="SAFETY STATISTICS", font=("Arial", 12, "bold"), 
                bg="#1e1e2f", fg="cyan").pack(pady=5)
        
        # Progress bars for statistics
        self.safety_meter = ttk.Progressbar(stats_frame, length=680, 
                                           mode='determinate', maximum=100)
        self.safety_meter.pack(pady=5)
        self.safety_label = tk.Label(stats_frame, text="Safety Level: 100%", 
                                    font=("Arial", 10), bg="#1e1e2f", fg="white")
        self.safety_label.pack()
        
        # Counters
        counters_frame = tk.Frame(stats_frame, bg="#1e1e2f")
        counters_frame.pack(pady=10)
        
        tk.Label(counters_frame, text="Total Frames:", font=("Arial", 10), 
                bg="#1e1e2f", fg="white").grid(row=0, column=0, padx=5)
        self.total_frames_label = tk.Label(counters_frame, text="0", font=("Arial", 10, "bold"), 
                                         bg="#1e1e2f", fg="white")
        self.total_frames_label.grid(row=0, column=1, padx=5)
        
        tk.Label(counters_frame, text="Safe Frames:", font=("Arial", 10), 
                bg="#1e1e2f", fg="white").grid(row=1, column=0, padx=5)
        self.safe_frames_label = tk.Label(counters_frame, text="0", font=("Arial", 10, "bold"), 
                                        bg="#1e1e2f", fg="green")
        self.safe_frames_label.grid(row=1, column=1, padx=5)
        
        tk.Label(counters_frame, text="Unsafe Events:", font=("Arial", 10), 
                bg="#1e1e2f", fg="white").grid(row=2, column=0, padx=5)
        self.unsafe_events_label = tk.Label(counters_frame, text="0", font=("Arial", 10, "bold"), 
                                          bg="#1e1e2f", fg="red")
        self.unsafe_events_label.grid(row=2, column=1, padx=5)
        
        # Alert controls
        self.alarm_enabled = tk.BooleanVar(value=True)
        tk.Checkbutton(stats_frame, text="Enable Alarms", variable=self.alarm_enabled, 
                      font=("Arial", 10), bg="#1e1e2f", fg="white", selectcolor="#121420").pack(pady=5)

        self.stop_flag = False
        self.alarm_on = False
        self.csv_file = open("unsafe_events_log.csv", mode='w', newline='')
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_writer.writerow(["Timestamp", "Label", "Frame Position"])

    def load_video(self):
        video_path = filedialog.askopenfilename()
        if video_path:
            # Reset statistics when loading new video
            global safety_stats
            safety_stats = {
                'total_frames': 0,
                'safe_frames': 0,
                'unsafe_events': 0,
                'last_alert': None,
                'alert_count': 0
            }
            self.update_stats_display()
            threading.Thread(target=self.process_video, args=(video_path,), daemon=True).start()

    def update_stats_display(self):
        total = max(1, safety_stats['total_frames'])
        safety_percent = (safety_stats['safe_frames'] / total) * 100
        
        self.safety_meter['value'] = safety_percent
        self.safety_label.config(text=f"Safety Level: {safety_percent:.1f}%")
        self.total_frames_label.config(text=str(safety_stats['total_frames']))
        self.safe_frames_label.config(text=str(safety_stats['safe_frames']))
        self.unsafe_events_label.config(text=str(safety_stats['unsafe_events']))
        
        # Change color based on safety level
        if safety_percent > 80:
            self.safety_meter['style'] = 'green.Horizontal.TProgressbar'
        elif safety_percent > 50:
            self.safety_meter['style'] = 'yellow.Horizontal.TProgressbar'
        else:
            self.safety_meter['style'] = 'red.Horizontal.TProgressbar'

    def log_event(self, message, label, frame_position):
        timestamp = time.strftime("%H:%M:%S")
        log_message = f"[{timestamp}] {message} (Label: {label}, Frame: {frame_position})"
        self.log_box.config(state='normal')
        self.log_box.insert(tk.END, log_message + "\n")
        self.log_box.see(tk.END)
        self.log_box.config(state='disabled')
        self.csv_writer.writerow([timestamp, label, frame_position])

    def play_alarm(self):
        if not self.alarm_on and self.alarm_enabled.get():
            self.alarm_on = True
            try:
                playsound("D:/beep.wav")
            except Exception as e:
                print("Alarm error:", e)
            self.alarm_on = False

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frame_buffer = deque(maxlen=sequence_len)
        frame_count = 0

        while cap.isOpened() and not self.stop_flag:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            safety_stats['total_frames'] += 1
            
            if frame_count % 2 != 0:
                continue

            display_frame = frame.copy()
            results, is_unsafe = is_unsafe_yolo(frame)
            
            # Draw all detections with improved visibility
            for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
                x1, y1, x2, y2 = map(int, box[:4])
                class_name = results.names[int(cls)]
                confidence = float(conf)
                
                # Set colors and thickness
                color = (0, 0, 255) if class_name in unsafe_labels_yolo else (0, 255, 0)
                box_thickness = 2
                text_thickness = 2
                font_scale = 0.8  # Increased font size
                
                # Draw bounding box
                cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, box_thickness)
                
                # Create text label
                label = f"{class_name} {confidence:.2f}"
                font = cv2.FONT_HERSHEY_SIMPLEX
                
                # Calculate text size
                (text_width, text_height), _ = cv2.getTextSize(label, font, font_scale, text_thickness)
                
                # Text background position (ensure it stays in frame)
                text_bg_x1 = x1
                text_bg_y1 = y1 - text_height - 10  # Don't go above top
                text_bg_x2 = x1 + text_width
                text_bg_y2 = y1

                if text_bg_y1 < 0:
                    text_bg_y1 = y1 + text_height + 10
                    text_bg_y2 = y1

                cv2.putText(display_frame, label, (x1, y1 - 5), 
                           font, font_scale, (0, 0, 0), text_thickness + 2)  # Black outline
                cv2.putText(display_frame, label, (x1, y1 - 5), 
                           font, font_scale, (255, 255, 255), text_thickness)  # White text
                
                # Draw filled background
                cv2.rectangle(display_frame, 
                             (text_bg_x1, text_bg_y1),
                             (text_bg_x2, text_bg_y2), 
                             color, -1)

            resized_frame = cv2.resize(display_frame, (720, 480))

            if is_unsafe:
                frame_buffer.append(frame)
    
                # Only analyze every 2nd unsafe frame when we have enough
                if len(frame_buffer) >= min_analysis_frames and frame_count % 3 == 0:
                    # Use actual frames only (no padding)
                    actual_frames = list(frame_buffer)
        
                    # If we must have exact sequence_len, use smarter sampling:
                    if len(actual_frames) >= sequence_len:
                        # Take every nth frame to get sequence_len frames
                        step = len(actual_frames) // sequence_len
                        analysis_frames = actual_frames[::step][:sequence_len]
                    else:
                        analysis_frames = actual_frames
            
                        pred_class = predict_unsafe_behavior(analysis_frames)
                    label = class_map.get(pred_class, "Unknown")

                    if pred_class != 0:
                        # Update statistics
                        safety_stats['unsafe_events'] += 1
                        
                        # Add main warning text
                        cv2.putText(resized_frame, f"UNSAFE: {label}", (10, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                        self.status_label.config(text=f"Status: {label} detected!", fg="red")
                        self.log_event(f"UNSAFE behavior detected", label, frame_count)
                        
                        # Trigger alarm (with cooldown)
                        current_time = time.time()
                        if (safety_stats['last_alert'] is None or 
                            (current_time - safety_stats['last_alert']) > 5):  # 5 second cooldown
                            threading.Thread(target=self.play_alarm, daemon=True).start()
                            safety_stats['last_alert'] = current_time
                            safety_stats['alert_count'] += 1
            else:
                safety_stats['safe_frames'] += 1
                self.status_label.config(text="Status: Safe", fg="green")
                frame_buffer.clear()

            # Update statistics display
            self.update_stats_display()
            
            # Convert for display in Tkinter
            frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
            self.video_label.update()

        cap.release()
        self.status_label.config(text="Status: Video Completed", fg="white")
        self.csv_file.close()

if __name__ == "__main__":
    root = tk.Tk()
    
    # Configure progressbar style
    style = ttk.Style()
    style.theme_use('clam')
    style.configure('green.Horizontal.TProgressbar', background='green')
    style.configure('yellow.Horizontal.TProgressbar', background='yellow')
    style.configure('red.Horizontal.TProgressbar', background='red')
    
    app = SafetyMonitorApp(root)
    root.mainloop()


0: 480x800 1 opened_panel, 1 safe_walkway, 3837.7ms
Speed: 267.4ms preprocess, 3837.7ms inference, 598.8ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 opened_panel, 1 safe_walkway, 458.2ms
Speed: 0.0ms preprocess, 458.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 opened_panel, 1 safe_walkway, 449.1ms
Speed: 15.6ms preprocess, 449.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 opened_panel, 1 safe_walkway, 484.3ms
Speed: 15.9ms preprocess, 484.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 opened_panel, 1 safe_walkway, 463.2ms
Speed: 15.6ms preprocess, 463.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 opened_panel, 1 safe_walkway, 420.6ms
Speed: 15.6ms preprocess, 420.6ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 opened_panel, 1 safe_walkway, 470.1ms
Speed: 15.6ms preprocess, 470.1ms inf